# Taking a Picture of a Tree but It's Windy and Its Leaves Are Randomly Falling
## Ryan Rickey
### VIS 142 FA 2022 - A05 - Coral Pereda

Start with generating an image:

In [ ]:
# code edited from https://pythonguides.com/fractal-python-turtle/
from turtle import *
import turtle

speed('fastest')

right(-90)
angle = 30

def yaxis(size, lvl):
    if lvl > 0:
        colormode(255)
        pencolor(255 // lvl**2, 255 // lvl, 255 // (lvl*3 % 255))
        forward(size)
        right(angle)
        yaxis(0.8 * size, lvl - 1)
        pencolor(0, 255 // lvl, 0)
        lt(2 * angle)
        yaxis(0.8 * size, lvl-1)
        pencolor(0, 255 // lvl, 0)
        right(angle)
        forward(-size)
yaxis(44, 10)
turtle.done()

See 'rickey_in1.png' for the result of this code.


In [3]:
from PIL import Image
import numpy
from numpy import asarray
import random
import png
from Chunk import Chunk

image = Image.open('rickey_in1.png')

treedata = asarray(image)
# print(treedata.shape)
# 450 x 512 original input size

# arbitrary size of output
width = 300
height = 300

imagedata = []
transform = 0

# flip a coin to blur image
blur = random.randint(0, 1)
if blur == 0:
    pad = random.randint(0, len(treedata[0])) % 100
# else reset pad in the loop
for h in range(height):
    row = []
    for w in range(width):
    	# reset pad each iteration to create a blur effect
        if blur == 1:
            pad = random.randint(0, len(treedata[0])) % 100
        for x in treedata[h,(w + pad + 40),:3]:
            if x == 0:
                transform = h + random.randint(0, h //2)
            row.append((x + transform) % 256)
    imagedata.append(row)


A function to check if we're going to go out of bounds of the image, and if so returns a new value that won't. Otherwise it keeps the value the same.

In [4]:
def round_dims(max_dim, start_dim, dim):
	if start_dim + dim > max_dim:
		return max_dim - start_dim
	return dim

This Chunk class stores four data points so we can draw the chunks more simply

In [5]:
class Chunk:
	h_start = 0
	w_start = 0
	height = 100
	width = 100
	
	def __init__(self, h_start, w_start, height, width):
		self.h_start = h_start
		self.w_start = w_start
		self.height = height
		self.width = width
	
	def draw(self):
		for h in range(height):
			for w in range(width):
				return 0


Now that I have a way to store the daa for these 'chunks,' make random ones!

In [11]:
# pick a random number of chunks
# can play with this number to see how long it take the rasbperry pi to run it
num_chunks = random.randint(1, 55)
print('number of chunks: %d' %num_chunks)

chunks = []
for i in range(num_chunks):
    # get a random starting pixel for the height
	rh_start = random.randint(0, height)
    # get a random starting pixel for the width
	rw_start = random.randint(0, width * 3)
    # get a random height for the chunk
	r_height = random.randint(1, height // 4)
    # get a random width for the chunk
	r_width = random.randint(1, width * 3 // 4)

    # to prevent the random h and w from going out of bounds
	r_height = round_dims(height, rh_start, r_height)
	r_width = round_dims(width*3, rw_start, r_width)

    # add each chunk to a list
	chunks.append(Chunk(rh_start, rw_start, r_height, r_width))

number of chunks: 19


Iterate through the chunks and draw them (or don't) on top of imagedata

In [12]:
# now iterate through chunks and draw them on top of imagedata
for chunk in chunks:
	for h in range(chunk.height):
		if h + chunk.h_start >= height: # safety
			break
		for w in range(chunk.width):
			if chunk.w_start + w >= width * 3 - 3: # safety
				break
			# flip a coin to decide whether to draw over imagedata
			transparent = random.randint(0, 1)
			if transparent == 1: # don't draw; continue
				w += 3
				continue
            # otherwise, draw!
			for p in range(3): # to change each R, G, B values
				# print('at [%d, %d]' %((chunk.h_start + h), (chunk.w_start + w)))
				val = random.randint(0, chunk.w_start // (chunk.h_start+1)) + h * w % 256
				imagedata[chunk.h_start + h][chunk.w_start + w + p] = random.randint(0, 255)


And of course, save the image!

In [13]:
# save image
chunkpng = png.from_array(imagedata, 'RGB').save('rickey_out1.png')